In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
np.random.seed(42)
n_sample=300

In [27]:
state=["Sunny","Cloudy","Rainy"]

In [29]:
transition={
    'Sunny': [0.7, 0.2, 0.1],
    'Cloudy': [0.3, 0.4, 0.3],
    'Rainy': [0.2, 0.3, 0.5]
}

In [31]:
emission={
    "Sunny" : lambda: [np.random.normal(30,2),np.random.normal(40,5)],
    'Cloudy': lambda: [np.random.normal(22, 2), np.random.normal(60, 5)],
    'Rainy': lambda: [np.random.normal(18, 2), np.random.normal(80, 5)]
}

In [33]:
weather=[]
observation=[]

In [35]:
cur_state="Sunny"
for i in range(n_sample):
    weather.append(cur_state)
    observation.append(emission[cur_state]())
    cur_state=np.random.choice(state,p=transition[cur_state])

In [39]:
df=pd.DataFrame(observation,columns=["temp","humidity"])
df['state']=weather

In [41]:
def create_discrete(col,bins):
    return pd.cut(col,bins=bins,labels=False)

In [49]:
df['temp_bin'] = create_discrete(df['temp'], bins=3)
df['hum_bin'] = create_discrete(df['humidity'], bins=3)
df['obs']=df['temp_bin']*3+df["hum_bin"]

In [51]:
obs=df['obs'].values

In [55]:
from hmmlearn.hmm import GaussianHMM,MultinomialHMM
model=MultinomialHMM(n_components=3,n_iter=100,random_state=42)

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [59]:
model.fit(obs.reshape(-1,1))

MultinomialHMM(n_components=3, n_iter=100,
               n_trials=array([3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 6, 6, 6, 3, 1, 6, 1, 2, 3,
       4, 4, 5, 1, 3, 6, 6, 6, 2, 2, 2, 2, 4, 2, 4, 4, 1, 2, 4, 4, 6, 6,
       6, 7, 6, 6, 6, 6, 6, 6, 2, 2, 2, 2, 4, 2, 2, 6, 6, 3, 1, 2, 2, 1,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 4, 6, 6, 6, 6, 6,
       6, 3, 4, 6, 6, 6, 6, 3, 6, 3, 3, 6, 7, 1, 6, 6, 6, 6, 2, 1, 6, 4,
       6, 4, 1, 2, 2, 2, 4, 6, 5, 1, 4, 2, 3, 1, 6, 6, 6, 3, 1, 6, 6, 6,
       4, 4, 6, 2, 5, 5, 2, 2, 4, 6, 6, 3, 2, 2, 2...
       6, 4, 2, 6, 4, 6, 6, 6, 6, 2, 4, 1, 3, 6, 6, 6, 3, 6, 6, 6, 1, 6,
       6, 4, 1, 1, 6, 6, 6, 6, 6, 1, 6, 3, 6, 6, 3, 2, 2, 4, 5, 1, 3, 6,
       3, 6, 6, 6, 3, 2, 2, 2, 4, 2, 4, 1, 4, 4, 4, 1, 6, 6, 3, 4, 2, 2,
       2, 2, 2, 1, 4, 4, 4, 6, 6, 2, 5, 4, 1, 1, 2, 4, 4, 4, 6, 1, 6, 6,
       6, 6, 6, 6, 6, 2, 2, 6, 6, 6, 6, 6, 4, 6, 3, 4, 6, 3, 3, 2, 2, 2,
       4, 6, 2, 4, 6, 4, 1, 1, 2, 1, 4, 7, 6, 6, 3, 4, 1, 2, 1, 6, 6, 4,
       4, 4, 6, 6, 2, 2, 3, 6, 6, 6, 6, 6, 2, 4], dtype=int64),
               random_state=RandomState(MT19937) at 0x228D00ABC40)

In [69]:
y_pred=model.predict(obs.reshape(-1,1))

In [81]:
from scipy.stats import mode
def map_states(true_state,pred_state):
    mapping={}
    for i,state in enumerate(np.unique(pred_state)):
        match=mode(true_state[pred_state==state],keepdims=True).mode[0]
        mapping[state]=match
    return [mapping[s] for s in pred_state]    

In [83]:
true_states=pd.Categorical(df['state']).codes
Prediction=map_states(true_states,y_pred)

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(true_states,Prediction)

0.48333333333333334

In [103]:
model_cont = GaussianHMM(n_components=3, covariance_type='full', n_iter=100, random_state=42)
model_cont.fit(df[['temp', 'humidity']])

C:\Users\kokar\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


GaussianHMM(covariance_type='full', n_components=3, n_iter=100, random_state=42)

In [105]:
hidden_states_cont = model_cont.predict(df[['temp', 'humidity']])

In [107]:
Prediction1=map_states(true_states,hidden_states_cont )

In [109]:
accuracy_score(Prediction1,true_states)

0.7566666666666667